In [1]:
import numpy as np
import matplotlib.pyplot as plt
from deepface.commons import functions
from deepface.basemodels import Facenet
import faiss
import glob

2022-12-22 12:06:21.885899: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 12:06:22.055691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-12-22 12:06:22.055719: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-22 12:06:23.292050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerr

In [2]:
model = Facenet.loadModel()

2022-12-22 12:06:24.956018: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-12-22 12:06:24.956049: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-22 12:06:24.956075: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-Zako): /proc/driver/nvidia/version does not exist
2022-12-22 12:06:24.956299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate comp

 enforce_detection tries to find a face in the image. If it cannot find a face, then function returns exception. 

In [3]:
representations = []
for face in glob.glob('/home/ubuntu/Desktop/Pre pics/*/*'):
    img = functions.preprocess_face(img=face, target_size=(160, 160),enforce_detection=False)
    embedding = model.predict(img)[0,:]
    representation = []
    representation.append(face)
    representation.append(embedding)
    representations.append(representation)

1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 61ms/step


In [4]:
embedding.shape

(128,)

In [5]:
print(len(representations))

444


In [6]:
embeddings = []
for i in range(0, len(representations)):
    embedding = representations[i][1]
    embeddings.append(embedding)
embeddings = np.array(embeddings, dtype='f')

In [7]:
embeddings.shape

(444, 128)

In [8]:
type(embeddings)

numpy.ndarray

In [9]:
dimensions = 128 # FaceNet output is 128 dimensional vector
 
metric = 'euclidean' # Euclidean Distance

if metric == 'euclidean':
    index = faiss.IndexFlatL2(dimensions)
elif metric == 'cosine':
        index = faiss.IndexFlatIP(dimensions)
        faiss.normalize_L2(embeddings)

In [10]:
print(embeddings)

[[-1.0857744  -0.5835091  -0.7929745  ...  0.38769963  0.26572952
   0.18438286]
 [-1.4374372  -0.26134783 -0.45117652 ...  1.1115645   0.5706582
   0.03774757]
 [-0.7575166  -0.10899252 -0.6374122  ...  1.330615   -0.17619456
  -0.2989119 ]
 ...
 [-1.179179    0.23319316 -0.7383041  ...  1.0589826   0.9468654
   0.6045745 ]
 [-1.1389638  -0.16273677 -0.6846086  ...  0.20248637 -0.49136022
   0.2360406 ]
 [-1.6203034   0.16980484 -0.47652665 ...  1.2434542   0.6072102
   0.12401131]]


In [11]:
index.add(embeddings)

In [12]:
#save
faiss.write_index(index,"vectors.index")